In [12]:
import os 
import cv2
from torch.utils.data import DataLoader , Dataset 
from torchvision import transforms

In [9]:
current_directory = os.getcwd()
os.chdir(current_directory)
datapath = r'Dataset\Pistachio_Image_Dataset\Pistachio_Image_Dataset'
subfolder = os.listdir(datapath)

images = []
labels = []
for sub in subfolder:
    label = subfolder.index(sub)
    path = os.path.join(datapath,sub)
    for img_path in os.listdir(path):
        image = cv2.imread(os.path.join(path,img_path))
        images.append(image)
        labels.append(label)

In [19]:
class Dataprep(Dataset):
    def __init__(self, features, labels, transforms = None):
        self.features = features
        self.labels = labels
        self.transforms = transforms
    
    def __getitem__(self, index) :
        image = self.features[index]
        label = self.labels[index]
        if self.transforms:
            image = self.transforms(image)

        return image, label
    
    def __len__(self):
        return len(self.labels)


data_transforms = transforms.Compose([transforms.ToTensor(), transforms.Resize((200,200))])
dataset = Dataprep(images,labels,data_transforms)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)
# data_sample = next(iter(data_loader))